# Stable Baselines3 Tutorial - Multiprocessing of environments
- How to use vectorized environments (multiprocessing).
- The tradeoff between speed and sample efficiency.

## About vectoized environments
A method to stack multiple independent environments into one. Benefits:
* Experience is collected more quickly.
* More diverse experience (range of states), improving exploration.



In [ ]:
# Dependencies: install swig

import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import gym

from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3 import PPO, A2C